In [1]:
import numpy as np
import pandas as pd

import beaapi
from beaapi.beaapi_error import no_data_err_msg, multiple_err_msg

In [2]:
# Get key from unversioned file
import os
from dotenv import load_dotenv
load_dotenv()
beakey = os.environ.get("beakey") #and be careful as USERID is passed back in the 'param' dict of many return values.

In [3]:
def gen_year_list(start, l, n):
    return [','.join([str(y) for y in np.arange(l)+start+i*l]) for i in range(n)]

In [22]:
# A quick version of what I do in pull_all.ipynb
do_checks = False
class_MNE = beaapi.get_parameter_values(beakey, 'MNE', 'Classification')
MNE_tbls = {}
my_err = None

print("DI")
years_dict_di = {("CountryByIndustry", "Outward"): gen_year_list(1997, 12, 2),
              ("CountryByIndustry", "Inward"): gen_year_list(1997, 12, 2),
              ("StatebyCountryofUBO","Outward"): gen_year_list(1997, 6, 4),
              ("StatebyCountryofUBO","Inward"): gen_year_list(1997, 6, 4)
              } # I manually check that there's no data before the start date
for cl_MNE in class_MNE["key"]:
    for direction in ["Outward", "Inward"]:
        years = years_dict_di.get((cl_MNE, direction),['All'])
        for year in years:
            #if cl_MNE=='CountryByIndustry' and direction=='Outward':
            #    continue
            try:
                bea_tbl = beaapi.get_data(beakey, "MNE", DirectionOfInvestment=direction, 
                                                                        Classification=cl_MNE, Year=year, GetFootnotes='Yes', 
                                                                        do_checks=do_checks)
                MNE_tbls[(cl_MNE, direction, year, 'DI')] = bea_tbl
            except beaapi.BEAAPIFailure as e:
                print("query_fail_str: cl_MNE=" + cl_MNE + ". direction=" + direction)
            except beaapi.BEAAPIResponseError as e:
                if e.args[0]==no_data_err_msg:
                    pass
                else:
                    raise e

print("AMNE")
years_dict_amne = {("StatebyCountryofUBO", "Outward"): gen_year_list(2009, 6, 2)}
for cl_MNE in class_MNE["key"]:
    for direction in ["Outward", "Inward", "State", "Parent"]:
        years = years_dict_amne.get((cl_MNE, direction),['All'])
        for year in years:
            try:
                bea_tbl = beaapi.get_data(beakey, "MNE", DirectionOfInvestment=direction, 
                                                OwnershipLevel="1", NonBankAffiliatesOnly="0", Classification=cl_MNE, Year=year, 
                                                GetFootnotes='Yes', do_checks=do_checks)
                MNE_tbls[(cl_MNE, direction, year, 'AMNE')] = bea_tbl
            except beaapi.BEAAPIFailure as e:
                print("query_fail_str: cl_MNE=" + cl_MNE + ". direction=" + direction)
            except beaapi.BEAAPIResponseError as e:
                if e.args[0]==no_data_err_msg or (e.args[0]==multiple_err_msg and no_data_err_msg in e.messages):
                    pass
                else:
                    raise e

DI
AMNE


In [35]:
# Consolidate tables
MNE_tbls2 = {}
to_consolidate = {}
for (cl_MNE, direction, year, mne_type), tbl in MNE_tbls.items():
    if year=='All':
        MNE_tbls2[(cl_MNE, direction, year, mne_type)] = tbl
    else:
        to_consolidate[(cl_MNE, direction, mne_type)] = to_consolidate.get((cl_MNE, direction, mne_type),[])+ [tbl]

for (cl_MNE, direction, mne_type), tbl_list in to_consolidate.items():
    MNE_tbls2[(cl_MNE, direction, 'All', mne_type)] = pd.concat(tbl_list, axis=0)

## Email 2

In [38]:
pd.set_option('display.max_rows', None)

In [39]:
for (cl_MNE, direction, year, mne_type),tbl in MNE_tbls2.items():
    df_small = tbl[['Row', 'RowCode']].drop_duplicates()
    df1 = df_small[df_small.duplicated('Row', False)]       
    if df1.shape[0]>0:
        print((cl_MNE, direction, mne_type))
        display(df1.sort_values('Row'))
        #display(tbl.merge(df1[['Row']].drop_duplicates(), on='Row').query("RowCode==' '").sort_values('Row'))

('Industry', 'Inward', 'DI')


,Row,RowCode
16452,other warehousing and storage,
20468,other warehousing and storage,4939
16366,petroleum storage for hire,
20396,petroleum storage for hire,4932


('StatebyCountryofUBO', 'Inward', 'DI')


,Row,RowCode
47052,georgia,337
8107,georgia,13
89420,other warehousing and storage,4939
64544,other warehousing and storage,
89402,petroleum storage for hire,4932
64506,petroleum storage for hire,


In [40]:
for (cl_MNE, direction, year, mne_type),tbl in MNE_tbls2.items():
    tbl['Row'] = tbl['Row'].str.lower()
    df_small = tbl[['Row', 'RowCode']].drop_duplicates()
    df2 = df_small[df_small.duplicated('RowCode', False)]
    if df2.shape[0]>0:
        print((cl_MNE, direction, mne_type))
        display(df2.sort_values('RowCode'))

('Country', 'Outward', 'DI')


,Row,RowCode
4000,other-europe,
5585,other-south america,
6383,other-central america,
7532,other-other western hemisphere,
8272,other-africa,
9042,other-middle east,
11316,other-asia and pacific,
11542,addenda:,
19972,east timor,660
21073,timor-leste,660


('Country', 'Inward', 'DI')


,Row,RowCode
3558,other-europe,
5228,other-south and central america,
6414,other-other western hemisphere,
6975,other-africa,
8461,other-middle east,
10763,other-asia and pacific,
10889,addenda:,
21697,tongo,
20283,east timor,660
21658,timor-leste,660


('CountrybyType', 'Inward', 'DI')


...

,Row,RowCode
2076,other-mining,
70395,"machinery, equipment, and supplies merchant wh...",
70164,other-wholesale trade,
54166,other-transportation equipment,
53666,other-machinery,
50924,addenda:,
50900,"machine shops; turned products; and screws, nu...",
57517,addendum:,
50196,other-asia and pacific,
36194,other-africa,


('StatebyCountryofUBO', 'Inward', 'DI')


,Row,RowCode
5024,other-europe,
93988,"nondepository credit intermediation, except br...",
74306,tongo,
75914,addendum:,
900,new england:,
2517,mideast:,
3938,great lakes:,
5330,plains:,
7173,southeast:,
10177,southwest:,


('StatebyCountryofUBO', 'Outward', 'AMNE')


,Row,RowCode
3604,other-mining,
77779,bakery products and tortillas,
108433,other warehousing and storage,
108403,petroleum storage for hire,
106718,"nondepository credit intermediation, except br...",
103088,"machine shop products, turned products, and sc...",
102908,cutlery and handtools,
84679,iron and steel mills,
81356,other-asia and pacific,
73938,"other-professional, scientific, and technical ...",


In [ ]:
#pd.set_option('display.max_rows', None)
display("Column -> multiple column codes")
for (cl_MNE, direction, year, mne_type),tbl in MNE_tbls2.items():
    df_small = tbl[['Column', 'ColumnCode']].drop_duplicates()
    df1 = df_small[df_small.duplicated('Column', False)]       
    if df1.shape[0]>0:
        tag = cl_MNE + "-" + direction + "-" + mne_type
        print(tag)
        display(df1.sort_values('Column'))
        #display(tbl.merge(df1[['Column']].drop_duplicates(), on='Column').query("ColumnCode==' '").sort_values('Column'))

display("Columns code -> multiple columns")
with pd.ExcelWriter("codes_to_mult_columns.xlsx") as writer: 
    for (cl_MNE, direction, year, mne_type),tbl in MNE_tbls2.items():
        df_small = tbl[['Column', 'ColumnCode']].drop_duplicates()
        df2 = df_small[df_small.duplicated('ColumnCode', False)]
        if df2.shape[0]>0:
            if cl_MNE == "IndustrybyCountryofUBO":
                cl_MNE = "IndbyCountryofUBO"
            tag = cl_MNE + "-" + direction + "-" + mne_type
            print(tag)
            df2 = df2.sort_values('ColumnCode')
            display(df2)
            df2.to_excel(writer, sheet_name=tag, index=False)
            #display(tbl.merge(df1[['Column']].drop_duplicates(), on='Column').query("ColumnCode==' '").sort_values('Column'))

display("Textually different column codes")
with pd.ExcelWriter("textually_different_colcodes.xlsx") as writer: 
    for (cl_MNE, direction, year, mne_type),tbl in MNE_tbls2.items():
        df_small = tbl[['ColumnCode']].drop_duplicates()
        df_small['ColumnCode_int'] = pd.to_numeric(df_small['ColumnCode'])
        df1 = df_small[df_small.duplicated('ColumnCode_int', False)]       
        if df1.shape[0]>0:
            if cl_MNE == "IndustrybyCountryofUBO":
                cl_MNE = "IndbyCountryofUBO"
            tag = cl_MNE + "-" + direction + "-" + mne_type
            print(tag)
            df1 = df1.sort_values('ColumnCode_int')
            display(df1)
            df1.to_excel(writer, sheet_name=tag, index=False)
            #display(tbl.merge(df1[['Column']].drop_duplicates(), on='Column').query("ColumnCode==' '").sort_values('Column'))